# Community Space Distribution and Attendance


## Introduction
In an era of less face-to-face interaction with increasing digital connectivity, the sense of community fostered by public spaces and gatherings gradually fades. Community Centers are one avenue of preservation nonetheless -- being public venues for socialization and recreational activity, their active function within a neighborhood can signify a close-knit community. Here, we seek to analyze the distribution of these centers as well as their respective attendance counts as a metric of community.

### Datasets Used
- [Community Center Attendance (WRDOC)](https://data.wprdc.org/dataset/daily-community-center-attendance)

Details the attendance for a number of community centers throughout Pittsburgh.

- [City of Pittsburgh Facilities (WRDOC)](https://data.wprdc.org/dataset/city-of-pittsburgh-facilities/resource/fbb50b02-2879-47cd-abea-ae697ec05170)

Lists a number of public facilities around Pittsburgh.

#### Density of Community Spaces
We will begin by importing pandas and geopandas to analyze the datasets.

In [1]:
import pandas as pd
import geopandas as gpd

# Loading neighborhood shape file.
shape_neighborhood = gpd.read_file('Datasets/neighborhoods/Neighborhoods_.shp')

shape_neighborhood.head(3)

,objectid,fid_blockg,statefp10,countyfp10,tractce10,blkgrpce10,geoid10,namelsad10,mtfcc10,funcstat10,...,shape_ar_1,page_numbe,plannerass,created_us,created_da,last_edite,last_edi_1,Shape__Are,Shape__Len,geometry
0,1,0.0,42,003,040500,1,420030405001,Block Group 1,G5030,S,...,7.843108e+06,15,Derek Dauphin,pgh.admin,2019-10-23T14:17:16.403Z,pgh.admin,2019-10-23T14:17:16.403Z,7.842997e+06,11525.904546,"POLYGON ((-79.95304 40.44203, -79.95302 40.442..."
1,2,1.0,42,003,040400,1,420030404001,Block Group 1,G5030,S,...,1.390463e+07,15,Derek Dauphin,pgh.admin,2019-10-23T14:17:16.403Z,pgh.admin,2019-10-23T14:17:16.403Z,1.390469e+07,20945.562570,"POLYGON ((-79.95455 40.45882, -79.95427 40.458..."
2,3,2.0,42,003,040200,2,420030402002,Block Group 2,G5030,S,...,5.999801e+06,15,Derek Dauphin,pgh.admin,2019-10-23T14:17:16.403Z,pgh.admin,2019-10-23T14:17:16.403Z,5.998649e+06,18280.484515,"POLYGON ((-79.96230 40.44294, -79.96220 40.442..."


With our libraries available for use, we will import our City of Pittsburgh Facilities dataset and name it "facilities". 

In [2]:
facilities = pd.read_csv('Datasets/city-facilities.csv')
facilities.head(3)

,_id,id,parcel_id,inactive,name,rentable,type,primary_user,address_number,street,...,neighborhood,council_district,ward,tract,public_works_division,pli_division,police_zone,fire_zone,latitude,longitude
0,1,650726265,120-J-300,f,57th Street Park Building,f,Storage,Department of Public Works,NaN,57TH ST,...,Upper Lawrenceville,7,10.0,42003101100,2.0,10.0,2.0,3-5,40.485666,-79.946450
1,2,783044037,2-H-284,f,Albert Turk Graham Park Shelter,f,Shelter,Department of Public Works,39.0,VINE ST,...,Crawford-Roberts,6,3.0,42003030500,3.0,3.0,2.0,2-1,40.440458,-79.984104
2,3,1997158435,23-R-157,f,Allegheny Northside Senior Center and Hazlett ...,t,Senior,CitiParks,5.0,ALLEGHENY SQ E,...,Allegheny Center,1,22.0,42003562700,1.0,22.0,1.0,1-6,40.453099,-80.005343


You will note, from a preliminary list of 3 rows, that the data contains a number of columns as well as types of facilities irrelevant to the overarching metric. We would like to sift through the dataset and target recreational spaces instead. To understand what data we work with, we will first extract a list of facilty types to determine how to splice the set.

In [8]:
# List the different possible names for the 'type' column
print(facilities['type'].unique())

['Storage' 'Shelter' 'Senior' 'Pool' 'Utility' 'Activity' 'Restrooms'
 'Service' 'Concession' 'Dugout' 'Pool/Rec' 'Rec Center' 'Office'
 'Pool Closed' 'Firehouse' 'Community' 'Vacant' 'Cabin' 'Medic Station'
 'Training' 'Police' 'Salt Dome' 'Recycling' 'SERVICE' 'STORAGE' 'POLICE'
 'TRAINING' 'OFFICE']



We can now see what types of facilities the dataset has provided us! To determine the density of community spaces, we will extract data entries deemed types 'Pool', 'Pool/Rec', 'Rec Center', and 'Community' to their own library. 

To do this, we will create a query mask of our terms and apply them.


In [9]:
spaces = ['Pool', 'Pool/Rec', 'Rec Center', 'Community']
typemask = facilities['type'].isin(spaces)
filtered = facilities[typemask]
filtered.sample(3)

,_id,id,parcel_id,inactive,name,rentable,type,primary_user,address_number,street,...,neighborhood,council_district,ward,tract,public_works_division,pli_division,police_zone,fire_zone,latitude,longitude
341,342,914518105,8-B-150,f,Sue Murray Filter Building,f,Pool,CitiParks,301.0,CEDAR AVE,...,Allegheny Center,1,22.0,42003562700,1.0,22.0,1.0,1-6,40.451086,-80.001775
6,7,2110346447,13-K-314,f,Arlington Pool Restrooms,f,Pool,CitiParks,1630.0,STERLING ST,...,South Side Slopes,3,16.0,42003160800,3.0,16.0,3.0,4-22,40.417753,-79.974688
328,329,1690301717,42-F-10,f,Sheraden Pool Building,f,Pool,CitiParks,1071.0,ADON ST,...,Sheraden,2,20.0,42003562500,5.0,20.0,6.0,1-16,40.457296,-80.059756


This query mask has allowed us to create a filtered data frame of community spaces. Still, to tidy up the data even further, let's form an entirely new frame with only the data we need -- we will include the name of the community space as well as its type and coordinates.

In [14]:
facil = pd.DataFrame(columns = ['Name', 'Type', 'Latitude', 'Longitude'])
facil['Name'] = filtered['name']
facil['Type'] = filtered['type']
facil['Latitude'] = filtered['latitude']
facil['Longitude'] = filtered['longitude']

print(len(facil))
facil.head(10)


41


,Name,Type,Latitude,Longitude
3,Ammon Recreation Center,Pool,40.448735,-79.977856
6,Arlington Pool Restrooms,Pool,40.417753,-79.974688
16,Banksville Pool Building and Shelter,Pool,40.414784,-80.040176
19,Bloomfield Pool and Recreation Center,Pool/Rec,40.462133,-79.953589
29,Brighton Heights Pool Building,Pool,40.486857,-80.030997
30,Brighton Heights Pool Guard Room,Pool,40.486976,-80.031012
39,Brookline Pool Building,Pool,40.390651,-80.007492
40,Brookline Pool Guard Room,Pool,40.390749,-80.007548
41,Brookline Recreation Center,Rec Center,40.391250,-80.008373
47,Burgwin Pool Building,Pool,40.405000,-79.937155


This is a much cleaner representation! We will now seek to determine the distribution of these 41 facilities to see which neighborhoods contain more spaces.